In [16]:
import numpy as np
from datasets import Dataset

seq_len, dataset_size = 512, 512

dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 2, (dataset_size))
}

ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [15]:
!pip install datasets

To print summary statistics for the GPU utilization and the training run with the Trainer we define two helper functions:

In [18]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summury(result):
  print(f"Time: {result.metrics['train_runtime']:.2f}")
  print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
  print_gpu_utilization()


In [17]:
!pip install pynvml

In [19]:
print_gpu_utilization()

GPU memory occupied: 1653 MB.


In [20]:
import torch

In [21]:
torch.ones((1024, 1024, 256)).to("cuda")
print_gpu_utilization()

GPU memory occupied: 2677 MB.


In [22]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-large-uncased").to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
print_gpu_utilization()

GPU memory occupied: 2943 MB.


262MB 정도 차지하는 군

In [25]:
 !nvidia-smi

Thu Jan 30 16:22:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             31W /   70W |    2678MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [32]:
default_args = {
    "output_dir": "tmp",
    "eval_strategy": "no",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
}

In [33]:
from transformers import TrainingArguments, Trainer, logging

In [34]:
logging.set_verbosity_error()

불필요 경고 메시지 출력 Xx

In [35]:
training_args = TrainingArguments(per_device_train_batch_size=4, **default_args)
trainer = Trainer(model=model, args=training_args, train_dataset=ds)
result = trainer.train()

{'train_runtime': 192.4946, 'train_samples_per_second': 2.66, 'train_steps_per_second': 0.665, 'train_loss': 0.7224196195602417, 'epoch': 1.0}


In [36]:
print_summury(result)

Time: 192.49
Samples/second: 2.66
GPU memory occupied: 8183 MB.
